# Neural Network From Scratch

In this notebook I will be creating Neural Network from scratch. All the material and references are from `DeepLearning.Ai` Specialization Course.

First I will import the numpy library

In [1]:
import numpy as np

We will create a one hidden layer network.

It will have `n_x` input, `n_h` nodes in the hidden layer and 1 output .

In [4]:
def initialize_wights(n_x,n_h):
    # Weight of the W1 will be (n_h,n_x)
    # Multiplied with 0.01 to make it near zero and help in training
    W1 = np.random.randn(n_h,n_x) * 0.01
    b1 = np.random.randn(n_h,1) * 0.01
    W2 = np.random.randn(1,n_h) * 0.01
    b2 = np.random.randn(1,1) * 0.01

    parameters = {
        "W1" : W1,
        "b1" : b1,
        "W2" : W2,
        "b2" : b2
    }

    return parameters


In [9]:
def sigmoid(x):
    output = 1 / (1 + np.exp(-x))
    return output

In [10]:
def relu(x):
    output = np.maximum(0,x)
    return output

In [21]:
def forward_pass(X , parameters) :
    z1 = np.dot(parameters['W1'],X) + parameters['b1']
    a1 = relu(z1)
    z2 = np.dot(parameters['W2'], a1) + parameters['b2']
    a2 = sigmoid(z2)
    return a2

## Now for L layer Deep Neural Network

In [30]:
def initialize_wights_deep(layer_list):

    # layer_list contains the info of the neural network
    # It can be like [3,4,5,1]
    # This means the input is 3, there are two hidden layers with 4 and 5 nodes respectively and 1 output

    parameters = {}
    no_layers = len(layer_list)

    for l in range(1, no_layers) :
        parameters["W" + str(l)] = np.random.randn(layer_list[l],layer_list[l-1]) * 0.01
        parameters["b" + str(l)] = np.random.randn(layer_list[l],1) * 0.01
        
    return parameters


Now doing forward pass in deep layers

In [38]:
def deep_forward(X,parameters):
    no_weights = len(parameters)//2
    A_temp = X
    for l in range(1,no_weights):
        z = np.dot(parameters["W" + str(l)], A_temp) 
        a = relu(z)
        A_temp = a
    
    Z = np.dot(parameters["W" + str(no_weights)], A_temp)
    A = sigmoid(Z)
    return A

In [39]:
layer_list = [3,4,5,6,7,8,9,1]

parameters = initialize_wights_deep(layer_list)
x  = np.random.randn(3,1)

In [40]:
deep_forward(X= x , parameters= parameters)

array([[0.5]])

Cost Function is` mean square error`

$$ \text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 $$

where:
- $ n $ is the number of data points,
- $ y_i $ is the actual value of the $i$-th data point,
- $ \hat{y}_i $ is the predicted value of the $i$-th data point.

For a single datapoint (i.e., $ n = 1 $), the formula simplifies to:

$$ \text{MSE} = (y - \hat{y})^2 $$

where:
- $ y $ is the actual value,
- $ \hat{y} $ is the predicted value.

In [ ]:
def cost_function(Y, AL) :
    # Y is the target
    # AL is the predicted output
    error = Y - AL
    # we will use mean square error
    cost = (1/2) * (error)**2
    return cost

## Now lets convert all into Class Module

In [47]:
class NeuralNetwork():
    def __init__(self,layer_list = [1,3,1]) -> None:
        self.layer_list = layer_list
        self.parameters = self.initialize_wights_deep(self.layer_list)

    def sigmoid(self,x):
        output = 1 / (1 + np.exp(-x))
        return output  
        
    def relu(self,x):
        output = np.maximum(0,x)
        return output
    
    def initialize_wights_deep(self,layer_list):
        parameters = {}
        no_layers = len(layer_list)

        for l in range(1, no_layers) :
            parameters["W" + str(l)] = np.random.randn(layer_list[l],layer_list[l-1]) * 0.01
            parameters["b" + str(l)] = np.random.randn(layer_list[l],1) * 0.01
            
        return parameters

    def deep_forward(self,X):
        no_weights = len(self.parameters)//2
        A_temp = X
        for l in range(1,no_weights):
            z = np.dot(self.parameters["W" + str(l)], A_temp) 
            a = self.relu(z)
            A_temp = a
        
        Z = np.dot(self.parameters["W" + str(no_weights)], A_temp)
        A = self.sigmoid(Z)
        return A

    def cost_function(self,Y, AL) :
        # Y is the target
        # AL is the predicted output
        error = Y - AL
        # we will use mean square error
        cost = (1/2) * (error)**2
        return cost

In [48]:
model = NeuralNetwork(layer_list= [3,4,5,6,7,8,9,1])

In [49]:
X = np.random.randn(3,1)

In [50]:
AL = model.deep_forward(X= X)

In [51]:
model.cost_function(Y=1 , AL= AL)

array([[0.125]])